In [1]:
import os, sys
sys.path.append('..')

import io
import torch
import numpy as np
import numpy.random as npr
import hydra
import dill
from tqdm import tqdm

import plotly.io as pio
pio.renderers.default = 'jupyterlab'
from scipy.interpolate import griddata
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import imageio

from irrep_actions.gym_util.multistep_wrapper import MultiStepWrapper
from irrep_actions.env.pusht.pusht_keypoints_env import PushTKeypointsEnv
from irrep_actions.workflow.base_workflow import BaseWorkflow
from irrep_actions.utils import mcmc

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# CH SO2 Lmax 5
#checkpoint = '../data/outputs/2024.01.30/16.43.31_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0350-test_mean_score=0.866.ckpt'

# CH SO2 Lmax 3
#checkpoint = '../data/outputs/2024.01.30/06.16.01_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0400-test_mean_score=0.975.ckpt'

# CH skip
# checkpoint = '../data/outputs/2024.01.30/19.08.21_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/latest.ckpt'

#checkpoint = '../data/outputs/2024.02.01/21.53.16_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0800-test_mean_score=0.740.ckpt'
checkpoint = '../data/outputs/2024.02.02/02.39.45_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/latest.ckpt'

payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
device = torch.device('cuda:2')

In [3]:
cfg = payload['config']
cls = hydra.utils.get_class(cfg._target_)

workflow = cls(cfg)
workflow: BaseWorkflow
workflow.load_payload(payload, exclude_keys=None, include_keys=None)

policy = workflow.model
policy = policy.to(device)
policy.eval()

ImplicitPolicy(
  (normalizer): LinearNormalizer(
    (params_dict): ParameterDict(
        (obs): Object of type: ParameterDict
        (action): Object of type: ParameterDict
      (obs): ParameterDict(
          (offset): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
          (scale): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
          (input_stats): Object of type: ParameterDict
        (input_stats): ParameterDict(
            (max): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
            (mean): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
            (min): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
            (std): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:2)]
        )
      )
      (action): ParameterDict(
          (offset): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:2)]
          (scale): Parameter containing: [torch.c

In [8]:
def eval(num_eps, start_seed=100000, num_disp = 100, num_rot=360, sample_act=False, vid_out=True):
    pbar = tqdm(total=num_eps)
    max_rewards = [0] * num_eps
    for eps in range(num_eps):
        env = MultiStepWrapper(
            PushTKeypointsEnv(render_action=False, random_goal_pose=True), 
            n_obs_steps=2, 
            n_action_steps=1,
            max_episode_steps=500
        )
        env.seed(start_seed + eps)
        
        obs = env.reset()
        d = False
        B = 1
        plots = list()
        
        while not d:
            Do = obs.shape[-1] // 2 
            obs = torch.from_numpy(obs[:, :Do].astype(np.float32).reshape(B,2,-1,2)).to(device) # 1x2x19x2
            x_obs = (obs.reshape(1,38,2)[:,:,0] - 255.0)
            y_obs = (obs.reshape(1,38,2)[:,:,1] - 255.0) * -1.0
            new_d = torch.concatenate((x_obs.unsqueeze(-1), y_obs.unsqueeze(-1)), dim=-1).view(1, -1).view(1,2,19*2)
            nobs = policy.normalizer['obs'].normalize(new_d)
            
            action_stats = policy.get_action_stats()
            action_dist = torch.distributions.Uniform(
                low=action_stats["min"], high=action_stats["max"]
            )
            actions = action_dist.sample((1, policy.pred_n_samples, 1)).to(
                dtype=nobs.dtype
            )
            if True:
                mag = torch.linspace(-1.0, 1.0, num_disp)
                mag = mag.view(1, -1).repeat(B, 1).view(B, -1, 1, 1).to(device)
                theta = torch.linspace(0, 2*np.pi, num_rot).to(device)
        
                with torch.no_grad():
                    logits = policy.energy_model.get_energy_ball(nobs, mag).view(1, -1)
                action_probs = torch.softmax(logits, dim=-1).view(1, num_disp, num_rot)
            else:
                action_probs, actions = mcmc.iterative_dfo(
                    policy.energy_model,
                    nobs,
                    actions,
                    [action_stats['min'], action_stats['max']],
                    harmonic_actions=policy.harmoncic_actions,
                    normalizer=policy.normalizer
                )

            if not sample_act:
                flat_indexes = action_probs.flatten(start_dim=-2).argmax(1)
                idx = [divmod(idx.item(), action_probs.shape[-1]) for idx in flat_indexes][0]
                #idxs = torch.argmax(action_probs, dim=-1).unsqueeze(-1)
            else:
                flat_indexes = torch.multinomial(action_probs.flatten(start_dim=-2), num_samples=1, replacement=True)
                idx = [divmod(idx.item(), action_probs.shape[-1]) for idx in flat_indexes][0]
                #idxs = torch.multinomial(action_probs, num_samples=1, replacement=True)
                
            #actions = actions[torch.arange(B).unsqueeze(-1), idxs].squeeze(1)
            actions = torch.tensor([mag[0,idx[0],0,0], theta[idx[1]]])
            if True:
                #action_probs = action_probs.view(B, num_disp, num_rot)
                max_disp = torch.max(action_probs, dim=-1)[0]
                E = action_probs[0,torch.argmax(max_disp, dim=1).item()].cpu().numpy()
          
            mag = policy.normalizer["action"].unnormalize(actions)[0]
            #theta = policy.normalizer["action"].unnormalize(actions)[:,:,1]
            theta = actions[1]
            x = mag * torch.cos(theta)
            y = mag * torch.sin(theta)
            actions = torch.concat([x.view(B,1), y.view(B,1)], dim=1).unsqueeze(1)
            
            if vid_out:
                img = env.render('human')
                
                f = plt.figure(figsize=(10,3))
                ax1 = f.add_subplot(111)
                ax2 = f.add_subplot(141, projection='polar')
                ax1.imshow(img)
                ax2.plot(np.linspace(0, 2*np.pi, E.shape[0]), E)
                #ax2.set_rmax(np.max(E))
                #ax2.set_rticks(np.round(np.linspace(np.min(E), np.max(E), 5), 3))
                ax2.set_rticks(list())
                ax2.grid(True)
                
                ax2.set_title(f"R={mag.item():.3f}", va="bottom")
                #plt.show()
                io_buf = io.BytesIO()
                f.savefig(io_buf, format='raw')
                io_buf.seek(0)
                img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                             newshape=(int(f.bbox.bounds[3]), int(f.bbox.bounds[2]), -1))
                plots.append(img_arr)
                io_buf.close()
                plt.close() 
        
            x_act = actions[:,:,0]
            y_act = actions[:,:,1] * -1.0
            new_act = torch.concatenate((x_act, y_act), dim=-1).view(B,1,2)
            obs, r, d, _ = env.step(new_act.squeeze(0).cpu().numpy())
            max_rewards[eps] = max(r, max_rewards[eps])
        imageio.mimwrite(f'plots/lmax_5/{eps}.gif', plots)
        print(max_rewards[eps])
        pbar.update(1)
    return max_rewards

In [10]:
max_rewards = eval(num_eps=50, start_seed=100000, num_disp=5000, num_rot=360, sample_act=False)


  0%|                                                                                                                                                                                                                                                       | 0/50 [00:00<?, ?it/s]ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

  2%|████▋                                          

0.42161692481806534


xcb_connection_has_error() returned true
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

  4%|█████████▍                                                                                                                                                                                                                                  | 2/50 [06:16<2:45:48, 207.27s/it]

0.30660014692689797


xcb_connection_has_error() returned true
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

  6%|██████████████▏                                                                                                                                                                                                                             | 3/50 [07:46<2:00:29, 153.82s/it]

...

0.871500947482967


xcb_connection_has_error() returned true
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 49/50 [1:09:14<01:26, 86.69s/it]

0.0


xcb_connection_has_error() returned true
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:10:40<00:00, 84.81s/it]

0.4866987331254077


In [11]:
np.mean(max_rewards)

0.7014800811121353

In [7]:
max_rewards

[0.0,
 0.9755642923803712,
 0.7470516666695183,
 0.7508666429696732,
 0.8504570447709737,
 0.18994403805709945,
 0.44905936476497704,
 0.873952535060591,
 0.4878608410036477,
 0.0,
 0.8161425289258735,
 0.8415382483259563,
 0.8553956657355963,
 0.1512921282779004,
 0.8429074055156375,
 0.6507714143922633,
 0.8845871647100202,
 0.763132983986121,
 0.991195448610806,
 0.6498296057961624,
 0.46092772595338066,
 0.9792647381102233,
 0.4191971769024325,
 0.45188545999786794,
 0.10916624302382937,
 0.4470525361539421,
 0.806353210672097,
 0.2336531721346822,
 0.2297827123973671,
 0.9235274230046095,
 1.0,
 0.580637654942133,
 0.39209816927153096,
 0.6186885026189002,
 0.0,
 0.0,
 0.3188967210614624,
 0.6853336430575782,
 0.0,
 1.0,
 0.0,
 0.023070244484604027,
 0.587517590184933,
 0.80651537748738,
 0.0,
 0.5145094160163427,
 0.44990190767079635,
 0.661887483272518,
 0.46591857698874933,
 0.2982480683808616]